<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [13]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-01-00a7d003df5a'
PROJECT = 'cloud-training-demos'
REGION = 'us-east1'

In [14]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [15]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [6]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use ABS(MOD()) on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>






In [44]:
import google.datalab.bigquery as bq
training_query = """
SELECT * FROM
(
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000 
)
WHERE MOD(ABS(hashmonth),10) < 8 and rand() < 0.00045
"""
evaluating_query = """
SELECT * FROM
(
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000 
)
WHERE MOD(ABS(hashmonth),10) >= 8 and rand() < 0.0005
"""

In [45]:
training_df = bq.Query(training_query).execute().result().to_dataframe()
evaluating_df = bq.Query(evluating_query).execute().result().to_dataframe()
print(training_df.count(),evaluating_df.count())

weight_pounds      12262
is_male            12273
mother_age         12273
plurality          12273
gestation_weeks    12189
hashmonth          12273
dtype: int64 weight_pounds      2790
is_male            2793
mother_age         2793
plurality          2793
gestation_weeks    2769
hashmonth          2793
dtype: int64


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [49]:
def process_df(df):
    df = df.dropna()
    df['is_male'] = 'NA'
    df['plurality'] = df['plurality'].map({1:'Single',2:'Twins',3:'Triplets',4:'Quadruplets',5:'Quintuplets'})

In [ ]:
traindf = 

In [46]:
traindf=training_df.dropna()
traindf['is_male'] = 'NA'
traindf['plurality'] = training_df['plurality'].map({1:'Single',2:'Twins',3:'Triplets',4:'Quadruplets',5:'Quintuplets'})

evaldf=evaluating_df.dropna()
evaldf['is_male'] = 'NA'
evaldf['plurality'] = evaldf['plurality'].map({1:'Single',2:'Twins',3:'Triplets',4:'Quadruplets',5:'Quintuplets'})

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [47]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [48]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

  2767 eval.csv
 12182 train.csv
 14949 total
==> eval.csv <==
8.1681268071,NA,36,Single,39.0,8904940584331855459
7.62578964258,NA,26,Single,39.0,-7146494315947640619
9.0058834027,NA,29,Single,41.0,1451354159195218418
9.4137385874,NA,27,Single,40.0,-4740473290291881219
7.87491199864,NA,22,Single,38.0,4331749811015213748
6.87621795178,NA,35,Single,38.0,-260598435387740869
5.4343947583,NA,21,Twins,41.0,1451354159195218418
8.062304921339999,NA,20,Single,44.0,3572456083892950068
6.7902376696,NA,24,Single,40.0,-7146494315947640619
8.000575487979999,NA,35,Single,39.0,3572456083892950068

==> train.csv <==
8.2232423726,NA,30,Single,39.0,3095933535584005890
6.8122838958,NA,31,Single,36.0,3095933535584005890
5.89957013112,NA,38,Single,38.0,3095933535584005890
6.18617107172,NA,20,Single,37.0,3095933535584005890
5.7099725858,NA,25,Single,38.0,3095933535584005890
8.7303055752,NA,42,Single,38.0,3095933535584005890
7.62578964258,NA,31,Single,39.0,3095933535584005890
6.75055446244,NA,34,Single,39.0,3

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License